In [8]:
import sympy as sp

a11, a12 = sp.symbols('a11 a12', real=True)
a21, a22 = sp.symbols('a21 a22', real=True)
A = sp.Matrix(
    [
        [a11, a12],
        [a21, a22],
    ]
)

orth = (A.T @ A - sp.eye(2)).norm(ord='fro') ** 2

avec = sp.Matrix([a11, a12, a21, a22])



In [ ]:
orth

In [ ]:
grad = sp.Matrix([orth.diff(x) for x in avec])
grad

In [9]:
hess = sp.Matrix(
    [
        [grad[i].diff(avec[j]) for j in range(4)]
        for i in range(4)
    ]
)
hess

Matrix([
[12*a11**2 + 4*a12**2 + 4*a21**2 - 4,               8*a11*a12 + 4*a21*a22,               8*a11*a21 + 4*a12*a22,                           4*a12*a21],
[              8*a11*a12 + 4*a21*a22, 4*a11**2 + 12*a12**2 + 4*a22**2 - 4,                           4*a11*a22,               4*a11*a21 + 8*a12*a22],
[              8*a11*a21 + 4*a12*a22,                           4*a11*a22, 4*a11**2 + 12*a21**2 + 4*a22**2 - 4,               4*a11*a12 + 8*a21*a22],
[                          4*a12*a21,               4*a11*a21 + 8*a12*a22,               4*a11*a12 + 8*a21*a22, 4*a12**2 + 4*a21**2 + 12*a22**2 - 4]])

In [ ]:
hess.inv()

In [10]:
import sympy as sp
from sympy.utilities.codegen import codegen
def gen_rust(fn_name, expr):
    return codegen(
        (fn_name, expr),
        language="Rust",
        prefix="rust_code"
    )[0][1]

In [ ]:
for i, g in enumerate(grad):
    print(gen_rust(f"grad_{i}", g))
    print()

In [12]:
for i in range(hess.shape[0]):
    for j in range(hess.shape[1]):
        h = hess[i, j]
        print(gen_rust(f"affine_hess_{i}_{j}", h))
        print()

/*
 *                      Code generated with SymPy 1.13.2
 *
 *              See http://www.sympy.org/ for more information.
 *
 *                       This file is part of 'project'
 */

fn affine_hess_0_0(a11: f64, a12: f64, a21: f64) -> f64 {

    let out1 = 12*a11.powi(2) + 4*a12.powi(2) + 4*a21.powi(2) - 4;
    out1

}


/*
 *                      Code generated with SymPy 1.13.2
 *
 *              See http://www.sympy.org/ for more information.
 *
 *                       This file is part of 'project'
 */

fn affine_hess_0_1(a11: f64, a12: f64, a21: f64, a22: f64) -> f64 {

    let out1 = 8*a11*a12 + 4*a21*a22;
    out1

}


/*
 *                      Code generated with SymPy 1.13.2
 *
 *              See http://www.sympy.org/ for more information.
 *
 *                       This file is part of 'project'
 */

fn affine_hess_0_2(a11: f64, a12: f64, a21: f64, a22: f64) -> f64 {

    let out1 = 8*a11*a21 + 4*a12*a22;
    out1

}


/*
 *                      Code generated wit

## Contact
node -> affine

In [ ]:
import sympy as sp

# initial position
px, py = sp.symbols('px py', real=True)
ux, uy, vx, vy = sp.symbols('ux uy vx vy', real=True)

# 定义向量
u = sp.Matrix([ux, uy])
v = sp.Matrix([vx, vy])
p = sp.Matrix([px, py])

a11, a12 = sp.symbols('a11 a12', real=True)
a21, a22 = sp.symbols('a21 a22', real=True)
tx, ty = sp.symbols('tx ty', real=True)
A = sp.Matrix(
    [
        [a11, a12],
        [a21, a22],
    ]
)
t = sp.Matrix([tx, ty])

x = A @ p + t

q = sp.Matrix([tx, ty, a11, a12, a21, a22])


In [ ]:
uv = v - u
ux = x - u

# if uv.dot(ux) <= 0
ux_norm = ux.norm()
ux_norm.diff(q)

In [ ]:
vu = u - v
vx = x - v

# if vu.dot(vx) <= 0
vx_norm = vx.norm()
vx_norm.diff(q)

In [ ]:
uv = v - u
ux = x - u

dist = abs(sp.Matrix([[uv, ux]]).det()) / uv.norm()

dist.diff(q)